In [14]:
import openai
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
openai.api_key = open("apikey.txt", "r").read().strip()
OPENAI_API_KEY = openai.api_key

In [15]:
#read csv file and save it in a dataframe
adf = pd.read_csv('woodpecker_attributes.csv')
output_df = pd.DataFrame(columns=['Attribute', 'LongHints', 'Reverse_GPT_Test'])

In [20]:
#iterate through each row in adf and generate long hints
for index, row in adf.iterrows():
    #create another for loop that iterates 5 times to test the long hint generated by GPT
    for i in range(3):    
        prompt = "You are a definition providing bot tasked with generating definitions and example values of an online product attribute. Given the product attribute, generate a scientific definition along with a bulleted list of values for the attribute. Please only list the definition and values and nothing else."
        prompt += "\n\nAttribute: " + row['Attribute']
        response = openai.Completion.create(
            engine="text-davinci-002",
            prompt=prompt,
            temperature=0.95,
            max_tokens=350,
        )
        #read the 'text' key in the json response and save it in a variable called 'longhint_temp'
        longhint_temp = response['choices'][0]['text']
        #print(longhint_temp)
        #check GPT's understanding of this longhint
        reverse_prompt ="You are now reversing roles and becoming an attribute name providing bot. You are given an attribute definition with examples and you have to guess the attribute name. Please only answer with the attribute name, nothing else."
        reverse_prompt += "\n\nLong Hint: " + longhint_temp
        reverse_response = openai.Completion.create(
            engine="text-davinci-002",
            prompt=reverse_prompt,
            temperature=0.01,
            max_tokens=35,
        )
        #read the 'text' key in the json reverse_response and save it in a variable called 'attribute_name'
        attribute_name = reverse_response['choices'][0]['text']
        #print(attribute_name)

        #fuzzy match the attribute by Damerau–Levenshtein distance and comapare name generated by GPT with the actual attribute name
        #if the fuzzy match score is greater than 80, then save the long hint in the dataframe
        if fuzz.ratio(attribute_name, row['Attribute']) > 80:
            Reverse_GPT_Test="Pass"
            #print("Matched")
            row['LongHints'] = longhint_temp
            break
        else:
            #print("Not Matched")
            Reverse_GPT_Test="Fail"
            continue
    #append attribute name, longhint_temp and Reverse_GPT_Test in a new dataframe called output_df
    #print(longhint_temp)
    new_df = pd.DataFrame({'Attribute': row['Attribute'], 'LongHints': longhint_temp.strip(), 'Reverse_GPT_Test': Reverse_GPT_Test}, index=[0])
    output_df = pd.concat([output_df, new_df], ignore_index=True)

    #output_df = output_df.append({'Attribute': row['Attribute'], 'LongHints': longhint_temp.strip(), 'Reverse_GPT_Test': Reverse_GPT_Test}, ignore_index=True)
    #print(output_df)
    #save output_df in a csv file
    output_df.to_csv('Attribute_with_longhints.csv', index=False)
    #print attribute name 
    print(row['Attribute']," - long hint generated")

aaiaBrandID  - long hint generated
